In [5]:
#Import Dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import pymongo

## NASA Mars News

In [6]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/'

In [7]:
# Retrieve page with the requests module
response = requests.get(url)
#response.headers
#response.content

In [8]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')

In [9]:
# Examine the results, then determine element that contains sought info
#print(soup.prettify())

In [10]:
# First paragraph result returned for first article on page
news_paragraph = soup.find('div', class_="rollover_description_inner").text.strip()
news_paragraph

'Starting July 27, news activities will cover everything from mission engineering and science to returning samples from Mars to, of course, the launch itself.'

In [11]:
# First title result returned for first article on page
news_title = soup.find('div', class_="content_title").a.text.strip()
news_title

'NASA to Broadcast Mars 2020 Perseverance Launch, Prelaunch Activities'

## JPL Mars Space Images - Featured Image


In [12]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [13]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [14]:
# Scrape page into Soup
html = browser.html
image_soup = BeautifulSoup(html, 'html.parser')
image_elem = image_soup.find_all('div', class_='carousel_items')

In [15]:
# Iterate through the feature image botton
feature_image_botton = image_elem[0].find('a', class_="button fancybox")
feature_image_botton

<a class="button fancybox" data-description="The magnetic field of our Milky Way galaxy as seen by ESA's Planck satellite. This image was compiled from the first all-sky observations of polarized light emitted by interstellar dust in the Milky Way." data-fancybox-group="images" data-fancybox-href="/spaceimages/images/mediumsize/PIA18048_ip.jpg" data-link="/spaceimages/details.php?id=PIA18048" data-title="Magnetic Map of Milky Way" id="full_image">
					FULL IMAGE
				  </a>

In [16]:

# Find the website source of the image

browser.links.find_by_partial_text('FULL IMAGE').click()
browser.links.find_by_partial_text('more info').click()


In [17]:
#Delay the extraction of information since the website took several seconds to load completely
browser.is_element_present_by_css("figure.lede", wait_time=1)

True

In [18]:

# Store the html of the current website
image_html = browser.html

In [19]:

# Scrape page into Soup
image_soup = BeautifulSoup(image_html, 'html.parser')

In [20]:

# Retrieve the figure tag 
figure_tag= image_soup.find_all('figure', class_='lede')

In [21]:
# Retrieve the image source from figure tag
image_source = figure_tag[0].find('a')['href']


In [22]:
image_source

'/spaceimages/images/largesize/PIA18048_hires.jpg'

In [23]:

# Store the image url 

website_url = 'https://www.jpl.nasa.gov'
featured_image_url = website_url + image_source
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18048_hires.jpg'

## Mars Facts

In [24]:
import pandas as pd

In [25]:
# Use splinter to obtain the information from the website 
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
url = 'https://space-facts.com/mars/'
browser.visit(url)


In [29]:
# Use pandas to read the html table data on the page into a list of dictionaries
df = pd.read_html('https://space-facts.com/mars/')


In [30]:
# Read the first dictionary in the list into a pandas dataframe and name columns
mars_df = df[0]
mars_df.columns = ['Parameter', 'Value']

In [31]:

mars_df.set_index('Parameter', inplace=True)
mars_df

,Value
Parameter,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [33]:
# Convert the dataframe into an html table, strip the end of line newlines and 
# write the result to an html file to view 
fact_table = mars_df.to_html()
fact_table = fact_table.replace('\n', '')
fact_table

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Value</th>    </tr>    <tr>      <th>Parameter</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

In [34]:
# # Inspect the result in a browser
mars_df.to_html('table.html')


## Mars Hemispheres

In [35]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [36]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [37]:
# Get page html and make beautifulsoup object
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [38]:
# Get the html containing the titles and put into a list
title_list = soup.find_all('div', class_='description')
title_list

[<div class="description"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><h3>Cerberus Hemisphere Enhanced</h3></a><span class="subtitle" style="float:left">image/tiff 21 MB</span><span class="pubDate" style="float:right"></span><br/><p>Mosaic of the Cerberus hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. This mosaic is composed of 104 Viking Orbiter images acquired…</p></div>,
 <div class="description"><a class="itemLink product-item" href="/search/map/Mars/Viking/schiaparelli_enhanced"><h3>Schiaparelli Hemisphere Enhanced</h3></a><span class="subtitle" style="float:left">image/tiff 35 MB</span><span class="pubDate" style="float:right"></span><br/><p>Mosaic of the Schiaparelli hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. The images were acquired in 1980 during early northern…</p></div>,
 <div class="description"><a 

In [42]:
# Loop through the div objects and scrape titles and urls of hires images
# Initiate the list to store dictionaries
hemisphere_image_urls = []
for title in title_list:
    # Navigate browser to page then click on title link to hires image page
    browser.visit(url)
    browser.click_link_by_partial_text(title.a.h3.text)

    # Grab the destination page html and make into BeautifulSoup object
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    # Parse the hires image source(src) relative url then append to domain name
    # for absolute url 
    img_url_list = soup.find('img', class_='wide-image')
    img_url = f"https://astrogeology.usgs.gov{img_url_list['src']}"

    # Create dictionary with returned values and add dict to hemisphere_image_urls list
    post = {
            'title': title.a.h3.text,
            'image_url': img_url
            }
    hemisphere_image_urls.append(post)

In [43]:
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'image_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'image_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'image_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'image_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]

In [2]:
import matplotlib.pyplot as plt

In [4]:
plt.savefig('https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg')
plt.show()

OSError: [Errno 22] Invalid argument: 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'

<Figure size 432x288 with 0 Axes>